In [ ]:
from keras_unet_collection import models as kmodels
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler

from keras import utils
from keras import models, backend
from keras.optimizers import Adam
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, \
    UpSampling2D, BatchNormalization, Concatenate, Activation, Conv2DTranspose, \
    Add

import re
import matplotlib.pyplot as plt
import sys, gc, os
import glob
import random
from tqdm import tqdm
import tensorflow as tf

In [ ]:
##Network Architecture: R-Unet
def URES34NET(org_shape, iteration, n_ch, donly):
    # Input
    original = Input(shape=org_shape)

    # Encoding_layer1
    for ii in range(iteration[0]):
        if ii == 0:
            c1 = first_block(original, 16)
        else:
            c1 = simple_block_dil(c1, 16)
    concate1 = c1

    # Encoding_layer2
    for ii in range(iteration[1]):
        if ii == 0:
            c2 = down_block(c1, 32)
        else:
            c2 = simple_block_dil(c2, 32)
    concate2 = c2

    # Encoding_layer3
    for ii in range(iteration[2]):
        if ii == 0:
            c3 = down_block(c2, 64)
        else:
            c3 = simple_block_dil(c3, 64)
    concate3 = c3

    # Encoding_layer4
    for ii in range(iteration[3]):
        if ii == 0:
            c4 = down_block(c3, 128)
        else:
            c4 = simple_block_dil(c4, 128)
    concate4 = c4

    if donly:
        print("Down: RUnet34, Up: Unet")
        x = deconv_unet(c4, concate3, 64)
        x = deconv_unet(x, concate2, 32)
        x = deconv_unet(x, concate1, 16)

        decoded = Conv2D(n_ch, (3, 3), activation='sigmoid', padding='same')(x)

    else:
        print("Down: RUnet34, Up: RUnet34")
        # Decoding_layer1
        for ii in range(iteration[2]):
            if ii == 0:
                u1 = up_block(c4, concate3, 64)
                print(u1.shape)
            else:
                u1 = simple_block(u1, 64)

        # Decoding_layer2
        for ii in range(iteration[1]):
            if ii == 0:
                u2 = up_block(u1, concate2, 32)
            else:
                u2 = simple_block(u2, 32)

        # Decoding_layer3
        for ii in range(iteration[0]):
            if ii == 0:
                u3 = up_block(u2, concate1, 16)
            else:
                u3 = simple_block(u3, 16)

        decoded = Conv2D(n_ch, (3, 3), activation='softmax', padding='same')(u3)

    model = models.Model(inputs=original, outputs=decoded)

    return model